# Auto Generated Agent Chat: Using RetrieveChat for Retrieve Augmented Code Generation and Question Answering

RetrieveChat is a convesational framework for retrieve augmented code generation and question answering. In this notebook, we demonstrate how to utilize RetrieveChat to generate code and answer questions based on customized documentations that are not present in the LLM's training dataset. RetrieveChat uses the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (e.g., [Automated Task Solving with Code Generation, Execution & Debugging](https://github.com/microsoft/FLAML/blob/main/notebook/autogen_agentchat_auto_feedback_from_code_execution.ipynb)). Essentially,`RetrieveAssistantAgent` and  `RetrieveUserProxyAgent` implements a different auto reply mechanism corresponding to the RetrieveChat prompts.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [mathchat] option.
```bash
pip install flaml[retrievechat]
```

In [9]:
# %pip install flaml[retrievechat]~=2.0.0rc5

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [10]:
from flaml import autogen

config_list = autogen.config_list_from_json(
    env_or_file=".config.local",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-35-turbo",
            "gpt-3.5-turbo",
        }
    },
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 and gpt-3.5-turbo models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct agents for RetrieveChat

We start by initialzing the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for RetrieveAssistantAgent. The detailed instructions are given in the user message. Later we will use the `RetrieveUserProxyAgent.generate_init_prompt` to combine the instructions and a math problem for an initial prompt to be sent to the LLM assistant.

In [11]:
from flaml.autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from flaml.autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

autogen.ChatCompletion.start_logging()

# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. By default, it is set to "./docs". Here we generated the documentations from FLAML's docstrings.
# Navigate to the website folder and run `pydoc-markdown` and it will generate folder `reference` under `website/docs`.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "docs_path": "../website/docs/reference",
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "embedding_model": "all-mpnet-base-v2",
    },
)

### Example 1

Use RetrieveChat to help generate sample code and automatically run the code and fix errors if there is any.

Problem: Which API should I use if I want to use FLAML for a classification task and I want to train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [13]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(assistant, problem=code_problem, search_string="spark")  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

doc_ids:  [['doc_36', 'doc_40', 'doc_15', 'doc_22', 'doc_16', 'doc_51', 'doc_44', 'doc_41', 'doc_45', 'doc_14', 'doc_0', 'doc_37', 'doc_38', 'doc_9']]
Adding doc_id doc_36 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_15 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the form

your 131072x1 screen size is bogus. expect trouble


23/08/10 20:18:19 WARN Utils: Your hostname, DESKTOP-TTKT4BA resolves to a loopback address: 127.0.1.1; using 172.30.239.248 instead (on interface eth0)
23/08/10 20:18:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/10 20:18:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[I 2023-08-10 20:18:31,953] A new study created in memory with name: optuna
[I 2023-08-10 20:18:32,138] A new study created in memory with name: optuna


23/08/10 20:18:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


2023-08-10 20:18:43.621265: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 20:18:43.712270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 20:18:45.087890: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Time exceeded, canceled jobs



Best model: None
Best model's config: None
Best loss: inf
ragproxyagent (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
None

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------
exitcode: 0 (execution succeeded)
Code output: 
None

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------


### Example 2

Use RetrieveChat to answer a question that is not related to code generation.

Problem: Who is the author of FLAML?

In [14]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Who is the author of FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_36', 'doc_58', 'doc_40', 'doc_51', 'doc_4', 'doc_23', 'doc_52', 'doc_15', 'doc_14', 'doc_59', 'doc_2', 'doc_7', 'doc_29', 'doc_56', 'doc_30', 'doc_3', 'doc_55', 'doc_44', 'doc_20', 'doc_33']]
Adding doc_id doc_36 to context.
Adding doc_id doc_58 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_58 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_51 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
exactly `UPDATE CONTEXT`.
If user's intent is code generation, you mus

### Example 3

Use RetrieveChat to help generate sample code and ask for human-in-loop feedbacks.

Problem: how to build a time series forecasting model for stock price using FLAML?

In [15]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "how to build a time series forecasting model for stock price using FLAML?"
ragproxyagent.initiate_chat(assistant, problem=code_problem)

doc_ids:  [['doc_39', 'doc_46', 'doc_49', 'doc_36', 'doc_38', 'doc_51', 'doc_37', 'doc_58', 'doc_48', 'doc_40', 'doc_47', 'doc_41', 'doc_15', 'doc_52', 'doc_14', 'doc_60', 'doc_59', 'doc_43', 'doc_11', 'doc_35']]
Adding doc_id doc_39 to context.
Adding doc_id doc_46 to context.
Adding doc_id doc_49 to context.
Adding doc_id doc_36 to context.
Adding doc_id doc_38 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
R

INFO:flaml.autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/lijiang1/code/FLAML1/flaml/autogen/oai/completion.py", line 204, in _get_response
    response = openai_completion.create(**config)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/home/lijiang1/an

assistant (to ragproxyagent):

To build a time series forecasting model for stock price using FLAML, follow these steps:

1. Install the necessary packages.

```bash
pip install flaml
```

2. Import the required libraries.

```python
import numpy as np
import pandas as pd
from flaml import AutoML
```

3. Load the stock price data (assuming your data is in a pandas dataframe).

```python
# Replace with your own data loading method
dataframe = pd.read_csv("stock_price_data.csv")
```

4. Set up time column and label column.

```python
time_col = "timestamp"
label = "price"
```

5. Prepare training data.

```python
dataframe[time_col] = pd.to_datetime(dataframe[time_col])
dataframe = dataframe.set_index(time_col)
dataframe = dataframe.sort_index()

# If test data is needed, you can split dataframe into train_data and test_data here
```

6. Set up the AutoML instance and fit the model.

```python
automl = AutoML()
automl.fit(
    dataframe=dataframe,
    label=label,
    task="ts_forecast_r

### Example 4

Use RetrieveChat to answer a question and ask for human-in-loop feedbacks.

Problem: Is there a function named `tune_automl` in FLAML?

In [16]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "Is there a function named `tune_automl` in FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_36', 'doc_40', 'doc_15', 'doc_14', 'doc_52', 'doc_51', 'doc_58', 'doc_21', 'doc_27', 'doc_35', 'doc_23', 'doc_12', 'doc_59', 'doc_4', 'doc_56', 'doc_47', 'doc_53', 'doc_20', 'doc_29', 'doc_33']]
Adding doc_id doc_36 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_15 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any pa

### Example 5

Use RetrieveChat to answer questions for [NaturalQuestion](https://ai.google.com/research/NaturalQuestions) dataset.

We'll first create a new document collection based on all the context corpus, then we select some questions and answer them with RetrieveChat.


In [17]:
corpus_file = "https://huggingface.co/datasets/thinkall/NaturalQuestionsQA/resolve/main/corpus.txt"

# Create a new collection for NaturalQuestions dataset
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "docs_path": corpus_file,
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "natural-questions",
        "chunk_mode": "one_line",
        "embedding_model": "all-MiniLM-L6-v2",
    },
)

In [18]:
import json

# queries_file = "https://huggingface.co/datasets/thinkall/NaturalQuestionsQA/resolve/main/queries.jsonl"
first_5_queries = """{"_id": "ce2342e1feb4e119cb273c05356b33309d38fa132a1cbeac2368a337e38419b8", "text": "what is non controlling interest on balance sheet", "metadata": {"answer": ["the portion of a subsidiary corporation 's stock that is not owned by the parent corporation"]}}
{"_id": "3a10ff0e520530c0aa33b2c7e8d989d78a8cd5d699201fc4b13d3845010994ee", "text": "how many episodes are in chicago fire season 4", "metadata": {"answer": ["23"]}}
{"_id": "8a19db28aa9be7dbcab290077d4a3e320d87e8badce1f7417f56f3720e527696", "text": "who sings love will keep us alive by the eagles", "metadata": {"answer": ["Timothy B. Schmit"]}}
{"_id": "06758a4918650a3f27398ff0520d4121c6d9cacfab3efcd84ddb83d2caa460ae", "text": "who is the leader of the ontario pc party", "metadata": {"answer": ["Patrick Walter Brown"]}}
{"_id": "d4a88ef2f705da63ced53932695505a2fc44879bafe6fd1224ffef9e9a590ad6", "text": "where did the last name keith come from", "metadata": {"answer": ["from Keith in East Lothian , Scotland", "from a nickname , derived from the Middle High German k\u012bt , a word meaning `` sprout '' , `` offspring ''"]}}
"""
queries = [json.loads(line) for line in first_5_queries.split("\n") if line]
questions = [q["text"] for q in queries]
answers = [q["metadata"]["answer"] for q in queries]
print(questions)
print(answers)

['what is non controlling interest on balance sheet', 'how many episodes are in chicago fire season 4', 'who sings love will keep us alive by the eagles', 'who is the leader of the ontario pc party', 'where did the last name keith come from']
[["the portion of a subsidiary corporation 's stock that is not owned by the parent corporation"], ['23'], ['Timothy B. Schmit'], ['Patrick Walter Brown'], ['from Keith in East Lothian , Scotland', "from a nickname , derived from the Middle High German kīt , a word meaning `` sprout '' , `` offspring ''"]]


In [19]:
for i in range(2):
    print(f"\n\n>>>>>>>>>>>>  Below are outputs of Case {i+1}  <<<<<<<<<<<<\n\n")

    # reset the assistant. Always reset the assistant before starting a new conversation.
    assistant.reset()

    # set `human_input_mode` to be `NEVER`
    ragproxyagent.human_input_mode = "NEVER"
    qa_problem = questions[i]
    ragproxyagent.initiate_chat(assistant, problem=qa_problem, n_results=3)



>>>>>>>>>>>>  Below are outputs of Case 1  <<<<<<<<<<<<




doc_ids:  [['doc_0', 'doc_3334', 'doc_720']]
Adding doc_id doc_0 to context.
Adding doc_id doc_3334 to context.
Adding doc_id doc_720 to context.
ragproxyagent (to assistant):

Adding doc_id doc_3334 to context.
Adding doc_id doc_720 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats 